# Ant-Fungi Hybrid Optimization Algorithm

Ant Colony Optimization (ACO) algorithm to incorporate some of the efficient path-finding strategies used by fungi (specifically slime mold or the mycelium network). This hybrid approach could leverage the speed of ants and the adaptive resilience of fungi, resulting in a potentially more powerful optimization algorithm.

Concept: Ant-Fungi Hybrid Optimization Algorithm (AFHO)
To design an algorithm that blends the strengths of both ACO and fungi-inspired methods, let's outline how this can be achieved:

## Ant-like Exploration and Pheromone Trails:

We'll maintain the classic ACO behavior where artificial ants traverse a network and deposit pheromones to communicate with each other, reinforcing the shortest paths.
Ants will prioritize paths with higher pheromone levels but also explore new paths with a certain probability, balancing exploitation and exploration.
Fungi-like Network Adaptation:

Introduce a mycelium-inspired adaptive network structure where nodes (representing possible paths) can grow, shrink, or retract based on their utility.
Use a feedback mechanism similar to how fungi reinforce nutrient-rich paths and retract from inefficient ones.
Incorporate dynamic path adjustment that allows for the network to change in real-time, adapting to new conditions or obstacles.
Hybrid Pheromone and Nutrient Flow:

Instead of just relying on pheromones, introduce a "nutrient flow" concept similar to how fungi transport nutrients through their network.
Paths that are frequently used by ants receive both pheromone reinforcement and nutrient flow, allowing for adaptive reinforcement.

Key Benefits of the Hybrid Approach
Speed and Adaptability: Combines the rapid path-finding capabilities of ants with the adaptive network resilience of fungi.
Dynamic Optimization: Adapts to changing environments and finds optimal solutions even if the landscape changes.
Scalability: Can be applied to large, complex networks like logistics, supply chain management, or transportation systems.
This approach leverages the best of both worlds to provide a robust solution for complex optimization problems.

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import sko
from sko.ACA import ACA_TSP


Dataset is divided into 7 tables, one table for all orders that needs to be assigned a route – OrderList table, and 6 additional files specifying the problem and restrictions. For instance, the FreightRates table describes all available couriers, the weight gaps for each individual lane and rates associated. The PlantPorts table describes the allowed links between the warehouses and shipping ports in real world. Furthermore, the ProductsPerPlant table lists all supported warehouse-product combinations. The VmiCustomers lists all special cases, where warehouse is only allowed to support specific customer, while any other non-listed warehouse can supply any customer. Moreover, the WhCapacities lists warehouse capacities measured in number of orders per day and the WhCosts specifies the cost associated in storing the products in given warehouse measured in dollars per unit.



Order ID is ID of the order made by the customer, product ID is the specific product ID customer ordered.

In [10]:
df = pd.read_excel('Supply chain logisitcs problem.xlsx')
df

,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.300000
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.940000
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.200000
3,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.160000
4,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.340000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9210,1.447305e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118
9211,1.447319e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118
9212,1.447322e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,245,0.294265
9213,1.447145e+09,2013-05-26,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683430,PLANT03,PORT09,278,2.480000
